In [1]:
import random
import time
import math
import csv
from queue import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 


In [2]:
def printBoard(board):
    print("   0    1    2")
    print("  ------------")
    idx = 0
    for i in range(3):
        print(i, end=" | ")
        for j in range(3):
            print(board[idx],end=" | ")
            idx += 1
        print("\n  ------------")

In [7]:
class Agent:
    def __init__(self, playerNumber):
        self.utilities = np.zeros(20000)
        self.ns = np.zeros(20000)
        self.alpha = 0.03
        self.epsilon = 0
        self.gamma = 0.05
        self.plays = 0
        self.totalPlays = 0
        self.gameCount = 0
        self.playerNumber = playerNumber
        self.lastState = None
        self.stateQueue = Queue()
        self.boardMultiplicator = 3 ** np.arange(9)
        self.winningLines = [
            [0, 3, 1],
            [3, 6, 1],
            [6, 9, 1],
            [0, 7, 3],
            [1, 8, 3],
            [2, 9, 3],
            [0, 9, 4],
            [2, 7, 2]
        ]
        self.perms = [
            [
                0, 1, 2,
                3, 4, 5,
                6, 7, 8
            ],
            [
                2, 5, 8,
                1, 4, 7,
                0, 3, 6
            ],
            [
                8, 7, 6,
                5, 4, 3,
                2, 1, 0
            ],
            [
                6, 3, 0,
                7, 4, 1,
                8, 5, 2
            ],
            [
                2, 1, 0,
                5, 4, 3,
                8, 7, 6
            ],
            [
                0, 3, 6,
                1, 4, 7,
                2, 5, 8
            ],
            [
                6, 7, 8,
                3, 4, 5,
                0, 1, 2
            ],
            [
                8, 5, 2,
                7, 4, 1,
                6, 3, 0
            ]
        ]
        self.permutations = [[3 ** i for i in perm] for perm in self.perms] 
    
    def setPlayerNumber(self, n):
        self.playerNumber = n
        
    def reward(self, playerNumber, board):
        winner = self.checkWinner(board)
        if winner == playerNumber:
            return 5
        elif winner != 0:
            return -1
        elif self.checkPossibleActions(board) == []:
            return -0.04
        return 0
        
    def checkWinner(self, board):
        winner = 0
        for player in range(1,3):
            for line in self.winningLines:
                for i in range(line[0], line[1], line[2]):
                    if board[i] == player:
                        winner = player
                        continue
                    else:
                        winner = 0
                        break
                if winner != 0: return winner
        return winner

    def checkPossibleActions(self, state):
        actions = []
        for i in range(len(state)):
            if state[i] == 0:
                actions.append(i)
        return actions

    def placeChip(self, tempState, chipLocation):
        tempState[chipLocation] = self.playerNumber
        return tempState

    def policy(self, state, utilities, epsilon, playerNumber):
        actions = self.checkPossibleActions(state)
        actionReward = -2
        currentAction = None
        if actions == []:
            return 0
        if random.random() < epsilon:
            return random.choice(actions)
        for action in actions:
            tempState = self.placeChip(np.copy(state), action)
            tempStateKey = self.hashStateFast(tempState)
            if tempStateKey in utilities:
                if utilities[tempStateKey] >= actionReward:
                    currentAction = action
                    actionReward = utilities[tempStateKey]
        if currentAction == None: 
            return random.choice(actions)
        return currentAction
    
    def hashStateFast(self, board):
        return np.sum(np.multiply(board, self.permutations[0]))

    def learningAgent(self, state):
        self.plays += 1
        self.epsilon = max(1 - (0.0000002 * self.totalPlays), 0.05)
        key = self.hashStateFast(state)
        self.utilities[key] = self.reward(self.playerNumber, state)
        if hasattr(self.lastState, 'shape'):
            lastBoardKey = self.hashStateFast(self.lastState)
            self.ns[lastBoardKey] += 1
            self.alpha = 60/(59 + self.ns[lastBoardKey])
            queueCount = 1
            for i in range(self.stateQueue.qsize()):
                queuedState = self.stateQueue.get()
                qStateKey = self.hashStateFast(queuedState)
                isInMatrix = qStateKey in self.utilities
                self.utilities[qStateKey] +=\
                                    (\
                                      self.alpha * \
                                      (\
                                          self.reward(self.playerNumber, queuedState)\
                                          + self.gamma\
                                          * self.utilities[key]\
                                          - self.utilities[qStateKey]\
                                      )\
                                    )\
#                                         / queueCount
                       
                for k in range(1, len(self.permutations)):
                    self.utilities[np.sum(np.multiply(\
                                                      self.permutations[k],\
                                                      queuedState\
                                                     ))] = self.utilities[qStateKey]
                queueCount += queueCount
                self.stateQueue.put(queuedState)
               
                    
        self.stateQueue.put(state)
        self.lastState = state
        if self.reward(self.playerNumber, state) != 0:
            self.gameCount += 1
            self.totalPlays += self.plays
            self.plays = 0
            self.stateQueue.queue.clear()
            self.lastState = None
            return 1
        action = self.policy(state, self.utilities, self.epsilon, self.playerNumber)
        return action

    def playGame(self, playerNumber, state):
        return self.placeChip(state, self.policy(state, self.utilities, 0, playerNumber))

In [8]:
winningLines = [
    [0, 3, 1],
    [3, 6, 1],
    [6, 9, 1],
    [0, 7, 3],
    [1, 8, 3],
    [2, 9, 3],
    [0, 9, 4],
    [2, 7, 2]
]
def checkWins(board):
    winner = 0
    for player in range(1,3):
        for line in winningLines:
            for i in range(line[0], line[1], line[2]):
                if board[i] == player:
                    winner = player
                    continue
                else:
                    winner = 0
                    break
            if winner != 0: return winner
    if (0 in board) == False:
        return 3
    return 0

def gameTurn(player):
    while True:
        print("Player", player)
        x = int(input("Choose y position: (0,1,2) "))
        y = int(input("Choose x position: (0,1,2) "))
        if x > 2 or y > 2:
            print("Out of bounds")
            continue
        if board[x,y] == 0:
            board[x,y] = player
            return checkWins(board)
        print("There is already a chip in this place")
    
    
def endGame(board, player1, player2):
    player1.learningAgent(board)
    player2.learningAgent(board)

def game():
    board = np.zeros((9,), dtype=int)
    winner = 0
    player1 = Agent(1)
    player2 = Agent(2)
    player1Move = None
    player2Move = None
    gamesDone = 0
    quit = True
    limit = 500000
    printCounter = 0
    while gamesDone < limit:
        if printCounter == 10000:
            print("Games done:", gamesDone)
            printCounter = 0
        while True:
            player1Move = player1.learningAgent(board)
            board =  player1.placeChip(board, player1Move)
            if gamesDone >= limit - 10:
                printBoard(board)
            if checkWins(board)!= 0:
                endGame(board, player1, player2)
                break
            player2Move = player2.learningAgent(board)
            board =  player2.placeChip(board, player2Move)
            if gamesDone >= limit - 10:
                printBoard(board)
            if checkWins(board)!= 0:
                endGame(board, player1, player2)
                break
        player1Move = player2Move = None
        board = np.zeros((9,), dtype=int)
        gamesDone += 1
        printCounter += 1
    return player1, player2



def playVsBot(player):
    quit = "n"
    board = np.zeros((9,), dtype=int)
    while quit != "y":
        while True:
            print("Enter the coordinates where you want to place your chip")
            x = input("X: ")
            y = input("Y: ")
            try:
                x = int(x)
                y = int(y)
            except ValueError:
                print("Values entered are not valid, try again")
                continue
            if board[y * 3 + x] == 0:
                board[y * 3 + x] = 1
            else:
                print("Theres already a chip there, try again")
                continue
            printBoard(board)
            if checkWins(board)!= 0:
                print("Game Over")
                break
            board = player.playGame(2, board)
            printBoard(board)
            if checkWins(board)!= 0:
                print("Game Over")
                break
        board = np.zeros((9,), dtype=int)
        quit = input("Quit (y/n)")

        
start = time.time()
player1, player2 = game()
end = time.time()
print("Time:", end - start)

#saving utilitity table to csv
# w = csv.writer(open("util.csv", "w"))
# for key, val in player2.utilities.items():
#     w.writerow([key, val])
    

playVsBot(player2)

    


Games done: 10000


KeyboardInterrupt: 

In [160]:
playVsBot(player2)
print(len(player2.utilities))

Enter the coordinates where you want to place your chip
X: 0
Y: 0
   0    1    2
  ------------
0 | 1 | 0 | 0 | 
  ------------
1 | 0 | 0 | 0 | 
  ------------
2 | 0 | 0 | 0 | 
  ------------
   0    1    2
  ------------
0 | 1 | 0 | 0 | 
  ------------
1 | 2 | 0 | 0 | 
  ------------
2 | 0 | 0 | 0 | 
  ------------
Enter the coordinates where you want to place your chip
X: 1
Y: 1
   0    1    2
  ------------
0 | 1 | 0 | 0 | 
  ------------
1 | 2 | 1 | 0 | 
  ------------
2 | 0 | 0 | 0 | 
  ------------
   0    1    2
  ------------
0 | 1 | 0 | 0 | 
  ------------
1 | 2 | 1 | 0 | 
  ------------
2 | 0 | 2 | 0 | 
  ------------
Enter the coordinates where you want to place your chip
X: 2
Y: 2
   0    1    2
  ------------
0 | 1 | 0 | 0 | 
  ------------
1 | 2 | 1 | 0 | 
  ------------
2 | 0 | 2 | 1 | 
  ------------
Game Over
Quit (y/n)y
3055
